In [2]:
#load in neccessary libraries 
import requests
import bs4
import pandas as pd 
import yfinance as yf 
import email 
import datetime 
from yahoo_fin import stock_info as si
import numpy as np

In [26]:
file_path = 'sagemaker-studiolab-notebooks/Access.txt'  #read in confidential access keys DO NOT PRINT 
with open(file_path, 'r') as file:
    content = file.read()

FileNotFoundError: [Errno 2] No such file or directory: 'sagemaker-studiolab-notebooks/Access.txt'

In [3]:
#function fetches stock symbols for U.S. stock exchanges and takes a single exchange as a parameter
def get_US_stock_list(exchange):
    headers = {
      'authority': 'api.nasdaq.com',
      'accept': 'application/json, text/plain, */*',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
      'origin': 'https://www.nasdaq.com',
      'sec-fetch-site': 'same-site',
      'sec-fetch-mode': 'cors',
      'sec-fetch-dest': 'empty',
      'referer': 'https://www.nasdaq.com/',
      'accept-language': 'en-US,en;q=0.9',
  }

    params = (
      ('tableonly', 'true'),
      ('limit', '25'),  
      ('offset', '0'),
      ('download', 'true'),
      ('exchange', exchange)
  )

    r = requests.get('https://api.nasdaq.com/api/screener/stocks', headers=headers, params=params)
    data = r.json()['data']
    df = pd.DataFrame(data['rows'], columns=data['headers'])
    df_filtered = df[~df['symbol'].str.contains("\.|\^|\s")]
    return df_filtered['symbol'].tolist()

In [21]:
#create a single list from the two largest US exchanges
stock_list_nasdaq = pd.DataFrame({'symbol' : get_US_stock_list('nasdaq')})
stock_list_nyse = pd.DataFrame({'symbol' : get_US_stock_list('nyse')})
stock_list_nasdaq.loc[:,'exchange'] = 'nasdaq'
stock_list_nyse.loc[:,'exchange'] = 'nyse'


stock_list = pd.concat([stock_list_nasdaq, stock_list_nyse])
stock_list.to_csv('datasets/stock_list.csv')

print('The nyse stock exchange contains ' + str(len(stock_list_nyse)) + ' stocks. The nasdaq stock exchange contains ' + 
      str(len(stock_list_nasdaq)) + ' stocks. Resulting in a total of ' + str(len(stock_list)) + ' stocks to detect from.')


The nyse stock exchange contains 2504 stocks. The nasdaq stock exchange contains 4279 stocks. Resulting in a total of 6783 stocks to detect from.


In [22]:
stock_list

,symbol,exchange
0,AACG,nasdaq
1,AACIW,nasdaq
2,AADI,nasdaq
3,AAL,nasdaq
4,AAME,nasdaq
...,...,...
2499,ZTR,nyse
2500,ZTS,nyse
2501,ZUO,nyse
2502,ZVIA,nyse
